In [1]:
import pandas as pd
from quantrl_lab.data import (
    DataSourceRegistry,
    DataProcessor,
    IndicatorRegistry,
    AlpacaDataLoader, 
    YfinanceDataloader, 
    AlphaVantageDataLoader
)
from quantrl_lab.feature_selection.indicator_analysis import (
    IndicatorAnalysis,
    IndicatorAnalysisConfig,
    analyze_single_indicator,
    rank_indicator_performance
)

from quantrl_lab.feature_selection.vectorized_strategy_factory import VectorizedStrategyFactory
from quantrl_lab.data.indicators.technical_indicators import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [2]:
print(f"Available indicators:  {IndicatorRegistry.list_all()}")

Available indicators:  ['SMA', 'EMA', 'RSI', 'MACD', 'ATR', 'BB', 'STOCH', 'OBV']


In [3]:
# Example
supported = VectorizedStrategyFactory.get_supported_indicators()
requirements = VectorizedStrategyFactory.get_indicator_requirements('MACD')

print(f"Supported indicators: {supported}")
print(f"Requirements for MACD: {requirements}")

Supported indicators: ['SMA', 'EMA', 'RSI', 'MACD', 'ATR', 'BB', 'STOCH', 'OBV']
Requirements for MACD: {'required': ['fast', 'slow', 'signal'], 'optional': []}


In [4]:
data_loader = DataSourceRegistry()

In [ ]:
olhcv_df = data_loader.get_historical_ohlcv_data(
    symbols="MU",
    start="2023-01-01",
    end="2025-01-01",
    timeframe="1d",
)

Fetching historical data for MU from 2023-01-01 to 2025-01-01 with timeframe 1d

In [6]:
indicator_configs = IndicatorAnalysis.create_indicator_configs(
    {
        'name': 'SMA',
        'param_grid': {'window': [5, 10, 20, 50]}
    },
    {
        'name': 'EMA',
        'param_grid': {'window': [5, 10, 20, 50]}
    },
    {
        'name': 'RSI',
        'param_grid': {'window': [7, 14, 21, 28]},
        'strategy_params': {'oversold': 30, 'overbought': 70}
    },
    {
        'name': 'BB',
        'param_grid': {'window': [20, 50], 'num_std': [2.0, 2.5]}, # Generates 4 configs
    },
    {
        'name': 'MACD',
        'params': [
            {'fast': 12, 'slow': 26, 'signal': 9},
            {'fast': 5, 'slow': 35, 'signal': 5}
        ]
    },
    {
        'name': 'OBV',
        'params': [{}] # For indicators with no parameters
    }
)


In [7]:
from rich import print
print(indicator_configs)

{
    'SMA_5': {'indicator_params': {'window': 5}, 'strategy_params': {}},
    'SMA_10': {'indicator_params': {'window': 10}, 'strategy_params': {}},
    'SMA_20': {'indicator_params': {'window': 20}, 'strategy_params': {}},
    'SMA_50': {'indicator_params': {'window': 50}, 'strategy_params': {}},
    'EMA_5': {'indicator_params': {'window': 5}, 'strategy_params': {}},
    'EMA_10': {'indicator_params': {'window': 10}, 'strategy_params': {}},
    'EMA_20': {'indicator_params': {'window': 20}, 'strategy_params': {}},
    'EMA_50': {'indicator_params': {'window': 50}, 'strategy_params': {}},
    'RSI_7': {'indicator_params': {'window': 7}, 'strategy_params': {'oversold': 30, 'overbought': 70}},
    'RSI_14': {'indicator_params': {'window': 14}, 'strategy_params': {'oversold': 30, 'overbought': 70}},
    'RSI_21': {'indicator_params': {'window': 21}, 'strategy_params': {'oversold': 30, 'overbought': 70}},
    'RSI_28': {'indicator_params': {'window': 28}, 'strategy_params': {'oversold': 30, 'overbought': 70}},
    'BB_20_2.0': {'indicator_params': {'window': 20, 'num_std': 2.0}, 'strategy_params': {}},
    'BB_20_2.5': {'indicator_params': {'window': 20, 'num_std': 2.5}, 'strategy_params': {}},
    'BB_50_2.0': {'indicator_params': {'window': 50, 'num_std': 2.0}, 'strategy_params': {}},
    'BB_50_2.5': {'indicator_params': {'window': 50, 'num_std': 2.5}, 'strategy_params': {}},
    'MACD_12_26_9': {'indicator_params': {'fast': 12, 'slow': 26, 'signal': 9}, 'strategy_params': {}},
    'MACD_5_35_5': {'indicator_params': {'fast': 5, 'slow': 35, 'signal': 5}, 'strategy_params': {}},
    'OBV': {'indicator_params': {}, 'strategy_params': {}}
}

In [8]:
settings = IndicatorAnalysisConfig(
    initial_capital=100000,
    transaction_cost=0.001,
    position_sizing='full'
)

comparison_df = rank_indicator_performance(
    data=olhcv_df,
    indicators=indicator_configs,
    config=settings
)

comparison_df

/Users/hanyuwu/Study/QuantRL-Lab/src/quantrl_lab/feature_selection/vectorized_strategy_implementations.py:84: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(SignalType.HOLD.value)
/Users/hanyuwu/Study/QuantRL-Lab/src/quantrl_lab/feature_selection/vectorized_strategy_implementations.py:84: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(SignalType.HOLD.value)
/Users/hanyuwu/Study/QuantRL-Lab/src/quantrl_lab/feature_selection/vectorized_strategy_implementations.py:84: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is

,indicator,total_return,annual_return,volatility,sharpe_ratio,max_drawdown,win_rate,profit_factor,sortino_ratio,calmar_ratio,final_value,total_trades
13,BB_20_2.5,0.648010,0.285027,0.236353,1.121316,-0.202972,0.129741,1.591049,0.763999,1.404264,164800.99118,20
8,RSI_7,2.170450,0.784673,0.756726,1.010503,-0.279018,0.249501,1.756396,2.068405,2.812267,317045.01906,19
3,SMA_50,1.451664,0.568579,0.890135,0.616287,-0.265825,0.313373,1.478263,2.355420,2.138918,245166.38820,30
17,MACD_5_35_5,0.438158,0.200101,0.304294,0.591864,-0.352165,0.239521,1.218901,0.595144,0.568202,143815.81023,56
18,OBV,1.182903,0.479764,0.792845,0.579892,-0.305033,0.249501,1.468598,1.483257,1.572829,218290.30119,70
11,RSI_28,0.280235,0.132032,0.309948,0.361453,-0.366186,0.197605,1.166090,0.372773,0.360559,128023.49811,1
0,SMA_5,0.282964,0.133243,0.323089,0.350500,-0.280734,0.249501,1.141707,0.455271,0.474622,128296.44513,130
4,EMA_5,0.281882,0.132762,0.323125,0.348975,-0.280141,0.265469,1.137601,0.462491,0.473913,128188.18350,130
5,EMA_10,0.225001,0.107245,0.346531,0.251767,-0.341748,0.277445,1.116896,0.304015,0.313814,122500.09728,96
16,MACD_12_26_9,0.152789,0.073984,0.321535,0.167895,-0.263852,0.239521,1.102431,0.176598,0.280400,115278.85575,38
